# DropPath (Stochastic Depth) — Baştan Sona Tanım, Kod, ve Modele Entegrasyonu

Bu notebook 3 şeyi yapar:

1) **DropPath’in ne olduğunu** en temelden ileri seviyeye açıklar (neden var, hangi problemi çözer, matematik).  
2) DropPath’in **kodunu adım adım inşa eder** (fonksiyonel → modül → schedule).  
3) DropPath’i bir **residual CNN modele entegre eder** ve entegrasyonu **model üzerinden açıklar**.

> Not: Kodlar “minimal ama gerçek” olacak şekilde seçildi. Amaç: DropPath mantığını net görmek.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## 0) Bu importlar ne işe yarıyor?

- `torch`: Tensor işlemleri, GPU kullanımı vb.
- `torch.nn`: Modül (layer) tanımlamak için sınıflar (Conv2d, BatchNorm2d, Module vb.)
- `torch.nn.functional`: Fonksiyonel API (loss, aktivasyon, bazı opsiyonlar)

Bu notebook’ta DropPath bir **nn.Module** olarak yazılacağı için `nn` kullanıyoruz.


\
## 1) En temel: Residual blok ne yapar?

Residual blok fikri:
\[
y = x + F(x)
\]

- \(x\): giriş (identity/skip yol)
- \(F(x)\): “öğrenen” yol (residual branch)

ResNet’in gücü burada:
- Eğer \(F(x)\) işe yaramazsa model yine de \(y \approx x\) ile stabil kalır.
- Derin ağlarda gradyan akışı daha sağlıklı olur.

DropPath tam bu yapıya oturur: **bazen \(F(x)\) yolunu kapatıp sadece \(x\) ile geçmek**.


In [ ]:
# Çok minimal bir residual ifade (sadece fikir)
def residual_forward(x, fx):
    return x + fx

### Bu kod bloğu ne anlatıyor?

- Bu bir “model” değil; sadece matematiksel fikri kodlaştırdık.
- `fx` burada \(F(x)\) üretimi gibi düşün.
- Çıktı: `x + fx` → residual toplama.


\
## 2) DropPath fikri: “Yolu bazen kapat”

DropPath eğitim sırasında şu işlemi yapar:

\[
y = x + \frac{m}{p}F(x), \quad m \sim \mathrm{Bernoulli}(p)
\]

- \(p\): **keep probability** (yolu açık tutma olasılığı)
- \(m\): 0 veya 1 üreten rastgele maske
  - \(m = 0\) → residual yol kapalı → \(y = x\)
  - \(m = 1\) → residual yol açık → \(y = x + \frac{1}{p}F(x)\)

Neden \(\frac{1}{p}\) ile ölçekliyoruz?
- Çünkü eğitimde bazen kapatıyoruz → beklenen değer düşer.
- \(\frac{1}{p}\) ile ölçekleyerek **beklenen (expected) aktivasyon seviyesini** koruyoruz.

**Inference (eval)** zamanında DropPath kapalıdır:
\[
y = x + F(x)
\]


## 3) DropPath’i en basit haliyle fonksiyon olarak yazalım

Önce “modül sınıfı” yazmadan önce, fikri fonksiyonla kurmak anlaşılır:
- training’de maske üret
- residual çıkışı `mask / keep_prob` ile çarp
- eval’de dokunma


In [3]:
def droppath_fn(x: torch.Tensor, drop_prob: float, training: bool) -> torch.Tensor:
    """Fonksiyonel DropPath.
    x: residual branch çıktısı (F(x))
    drop_prob: yolu kapatma olasılığı
    """
    if drop_prob == 0.0 or (not training):
        return x

    keep_prob = 1.0 - drop_prob

    # Batch-wise maske: her örnek için tek karar (0/1)
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # (B,1,1,1) veya (B,1)
    mask = torch.empty(shape, device=x.device, dtype=x.dtype).bernoulli_(keep_prob)

    # Beklenen değeri koru
    return x * mask / keep_prob


### Bu kod bloğu ne yapıyor? (satır satır mantık)

- `drop_prob == 0` veya `training=False` ise: **hiçbir şey yapmadan** `x` döndürür.
- `keep_prob = 1 - drop_prob`: yolun açık kalma olasılığı.
- `shape = (B,1,1,1,...)`: **batch bazlı** mask üretir.
  - Yani her örnek için “bu residual yol açık mı kapalı mı?” tek bir Bernoulli kararı.
- `bernoulli_(keep_prob)`: keep_prob olasılıkla 1 üretir.
- `x * mask / keep_prob`: kapalıysa sıfır, açıksa ölçekli residual döner.

Bu fonksiyonun önemli özelliği:
> DropPath **spatial bir maske üretmez**, feature map’in bir kısmını değil **residual yolun tamamını** etkiler.


In [4]:
# Quick sanity check
x = torch.randn(4, 64, 16, 16)
y_train = droppath_fn(x, drop_prob=0.5, training=True)
y_eval  = droppath_fn(x, drop_prob=0.5, training=False)

y_train.shape, (y_eval - x).abs().max().item()


(torch.Size([4, 64, 16, 16]), 0.0)

### Bu test ne gösterir?
- `y_train.shape` aynı kalır (DropPath şekli bozmaz).
- `training=False` iken `y_eval == x` (maks fark ~0) → inference’ta DropPath devre dışıdır.


## 4) DropPath’i nn.Module olarak yazalım (pratik kullanım)

Model içine tak-çıkar için DropPath genelde şu şekilde yazılır:
- `DropPath(drop_prob)` bir katman gibi durur
- `forward()` içinde training kontrolü yapar


In [5]:
class DropPath(nn.Module):
    """DropPath (Stochastic Depth) - path-level dropout."""
    def __init__(self, drop_prob: float = 0.0):
        super().__init__()
        self.drop_prob = float(drop_prob)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return droppath_fn(x, self.drop_prob, self.training)


### Bu kod bloğu ne yapıyor?

- `DropPath` bir **katman gibi** davranır.
- `self.training` PyTorch’un standart flag’ıdır:
  - `model.train()` → training=True
  - `model.eval()` → training=False
- `forward()` içinde az önce yazdığımız fonksiyona delegasyon yapıyoruz.

Bu yapı, DropPath’i model içinde şu şekilde kullanmanı sağlar:
```python
self.dp = DropPath(0.1)
out = self.dp(residual_out)
```


\
## 5) İleri seviye: DropPath oranını derinliğe göre schedule etmek

Pratikte DropPath genelde şu şekilde ayarlanır:

- Sığ bloklar: drop düşük
- Derin bloklar: drop yüksek

Lineer schedule örneği:
\[
drop(\ell) = d_{max} \cdot \frac{\ell}{L-1}
\]

- \(\ell\): blok index’i (0..L-1)
- \(L\): toplam blok sayısı
- \(d_{max}\): maksimum drop rate


In [6]:
def droppath_rate_at(block_idx: int, num_blocks: int, max_drop: float) -> float:
    if num_blocks <= 1:
        return 0.0
    return float(max_drop) * (block_idx / (num_blocks - 1))

# örnek: 6 blokta max_drop=0.2
rates = [droppath_rate_at(i, 6, 0.2) for i in range(6)]
rates


[0.0, 0.04000000000000001, 0.08000000000000002, 0.12, 0.16000000000000003, 0.2]

### Bu kod bloğu ne yapıyor?

- `block_idx=0` için drop ≈ 0 (sığ katmanlar stabil kalsın)
- `block_idx` arttıkça drop artar
- Son blokta drop ≈ `max_drop`

Bu schedule özellikle derin ResNet/Transformer’larda stabiliteyi artırır.


## 6) DropPath’i residual bloğa entegre edelim

En önemli kural:
> DropPath **residual branch çıktısına** uygulanır ve **toplama öncesinde** durur.

Yani:
1) \(F(x)\) hesaplanır  
2) `DropPath(F(x))` uygulanır  
3) `+ x` yapılır  


In [7]:
class BasicResBlock(nn.Module):
    """Conv-BN-Act -> Conv-BN -> DropPath -> +Skip -> Act"""
    def __init__(self, cin: int, cout: int, stride: int = 1, drop_path: float = 0.0):
        super().__init__()
        self.conv1 = nn.Conv2d(cin, cout, 3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(cout)
        self.act = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(cout, cout, 3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(cout)

        # DropPath residual yolun ÇIKIŞINA uygulanacak
        self.dp = DropPath(drop_path)

        self.skip = nn.Identity()
        if stride != 1 or cin != cout:
            self.skip = nn.Sequential(
                nn.Conv2d(cin, cout, 1, stride=stride, bias=False),
                nn.BatchNorm2d(cout),
            )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = self.skip(x)

        out = self.act(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        out = self.dp(out)  # <-- DropPath: residual branch'e uygulanır
        out = self.act(out + identity)
        return out


### Bu kod bloğu (model entegrasyonu) ne anlatıyor?

- `identity = self.skip(x)` → skip yolu (x veya 1×1 projeksiyon)
- `out = ...` → residual branch \(F(x)\) üretimi
- `out = self.dp(out)` → **DropPath burada**:
  - training’de bazen `out` sıfırlanır → blok “atlandı” gibi olur
  - eval’de dokunulmaz
  - açık kaldığında `1/keep_prob` ile ölçeklenir
- `out + identity` → residual toplama
- final `act` → blok çıkışı

Bu entegrasyon DropBlock’tan farklıdır:
- DropBlock: feature map’in **bazı bölgelerini** siler (spatial)
- DropPath: residual branch’in **tamamını** kapatır (structural)


## 7) Küçük bir modelde DropPath’i “blok bazlı schedule” ile kullanalım

Aşağıda CIFAR benzeri girişler için küçük bir ResNet türevi var:
- 6 adet residual blok
- DropPath oranı blok derinliğine göre lineer artıyor


In [8]:
class ResNetSmallDropPath(nn.Module):
    def __init__(self, num_classes: int = 100, max_drop: float = 0.1):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )

        # toplam blok sayısı: 6 (2 + 2 + 2)
        total_blocks = 6
        idx = 0

        # stage1 (32x32)
        dp1 = droppath_rate_at(idx, total_blocks, max_drop); idx += 1
        dp2 = droppath_rate_at(idx, total_blocks, max_drop); idx += 1
        self.stage1 = nn.Sequential(
            BasicResBlock(64, 64, stride=1, drop_path=dp1),
            BasicResBlock(64, 64, stride=1, drop_path=dp2),
        )

        # stage2 (16x16)
        dp3 = droppath_rate_at(idx, total_blocks, max_drop); idx += 1
        dp4 = droppath_rate_at(idx, total_blocks, max_drop); idx += 1
        self.stage2 = nn.Sequential(
            BasicResBlock(64, 128, stride=2, drop_path=dp3),
            BasicResBlock(128, 128, stride=1, drop_path=dp4),
        )

        # stage3 (8x8)
        dp5 = droppath_rate_at(idx, total_blocks, max_drop); idx += 1
        dp6 = droppath_rate_at(idx, total_blocks, max_drop); idx += 1
        self.stage3 = nn.Sequential(
            BasicResBlock(128, 256, stride=2, drop_path=dp5),
            BasicResBlock(256, 256, stride=1, drop_path=dp6),
        )

        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(256, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.stem(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        return self.head(x)


### Bu kod bloğu ne yapıyor? (entegrasyonu model üzerinden oku)

- `max_drop`: en derin blokta uygulanacak maksimum drop rate (örn 0.1)
- `droppath_rate_at(...)`: blok indeksi arttıkça drop rate artar
- Her `BasicResBlock` kendi `drop_path` oranıyla `DropPath` içerir:
  - `BasicResBlock(... drop_path=dp_k)`
- Böylece:
  - erken bloklar ≈ 0’a yakın drop (stabil öğrenme)
  - geç bloklar daha yüksek drop (regularization)

Bu modelin “DropPath’i entegre etme” şekli:
> DropPath her residual bloğun içinde, residual branch çıktısına uygulanarak yapılır.


## 8) Hızlı Çalışma Testi (Shape + Train/Eval farkı)

In [9]:
model = ResNetSmallDropPath(num_classes=100, max_drop=0.2)
x = torch.randn(8, 3, 32, 32)

model.train()
y_train = model(x)

model.eval()
y_eval = model(x)

y_train.shape, y_eval.shape


(torch.Size([8, 100]), torch.Size([8, 100]))

### Bu test neyi gösterir?
- Train ve eval modda çıktı şekli aynıdır.
- Train modda DropPath aktif olduğu için `y_train` deterministik değildir (her forward’da mask değişebilir).
- Eval modda DropPath kapalıdır, model deterministik davranır.
